In [1]:
import os
import random
import cProfile
import timeit

from cities.queries.causal_insight import CausalInsight
from cities.utils.data_grabber import (DataGrabber, list_interventions,
                                       list_outcomes)

smoke_test = "CI" in os.environ
smoke_test = True
num_samples = 10 if smoke_test else 1000

In [2]:
interventions = list_interventions()
outcomes = list_outcomes()
shifts = [1, 2, 3]

outcome = "unemployment_rate"
outcome2 = "gdp"
intervention = "spending_commerce"
intervention2 = "spending_transportation"
intervened_value = 0.7

data = DataGrabber()
data.get_features_wide(["gdp"])
gdp = data.wide["gdp"]
fips = gdp["GeoFIPS"][5] #1011
fips2 = gdp["GeoFIPS"][6] #1013


In [3]:
def basic_run():
    ci = CausalInsight(
        outcome_dataset=outcome,
        intervention_dataset=intervention,
        num_samples=num_samples,
        smoke_test=smoke_test,
    )

    ci.generate_tensed_samples()
    ci.get_fips_predictions(intervened_value=intervened_value, fips=fips)
    ci.plot_predictions(range_multiplier=1)

In [4]:
profiler_basic = cProfile.Profile()

profiler_basic.enable()
basic_run()
profiler_basic.disable()

# confirming that the time consuming moves involve the model
# gettign samples, etc.
profiler_basic.print_stats(sort='cumulative')

# note: restricting sites to tau drops time from 8s to 5s. 


True True


         939234 function calls (909466 primitive calls) in 1.240 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    1.241    0.620 interactiveshell.py:3514(run_code)
    792/2    0.001    0.000    1.241    0.620 {built-in method builtins.exec}
        1    0.001    0.001    1.241    1.241 433866942.py:1(<module>)
        1    0.000    0.000    1.240    1.240 4044749442.py:1(basic_run)
        1    0.000    0.000    0.841    0.841 causal_insight.py:107(generate_tensed_samples)
        3    0.001    0.000    0.808    0.269 causal_insight.py:65(generate_samples)
      132    0.001    0.000    0.639    0.005 readers.py:848(read_csv)
      132    0.003    0.000    0.636    0.005 readers.py:574(_read)
        3    0.003    0.001    0.610    0.203 modeling_utils.py:20(prep_wide_data_for_inference)
      132    0.001    0.000    0.541    0.004 readers.py:1732(read)
        9    0.001    0.000    0.478 

In [5]:
# this is prep pipeline
print(smoke_test)
ci = CausalInsight(
        outcome_dataset=outcome,
        intervention_dataset=intervention2,
        num_samples=num_samples,
        smoke_test=smoke_test
    )
ci.generate_tensed_samples()


True
True True


In [7]:
# this is the slim execution 
def slim_run():
    ci = CausalInsight(
        outcome_dataset=outcome,
        intervention_dataset=intervention,
        num_samples=num_samples,
    )

    ci.get_tau_samples()
    ci.get_fips_predictions(intervened_value=intervened_value, fips=fips)
    ci.plot_predictions(range_multiplier=1)

profiler_slim = cProfile.Profile()

profiler_slim.enable()
slim_run()
profiler_slim.disable()


profiler_slim.print_stats(sort='cumulative')


         249257 function calls (235951 primitive calls) in 0.509 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.509    0.255 interactiveshell.py:3514(run_code)
        2    0.000    0.000    0.509    0.255 {built-in method builtins.exec}
        1    0.001    0.001    0.509    0.509 207891617.py:2(slim_run)
       50    0.001    0.000    0.416    0.008 readers.py:848(read_csv)
       50    0.002    0.000    0.415    0.008 readers.py:574(_read)
        1    0.001    0.001    0.408    0.408 causal_insight.py:127(get_fips_predictions)
       50    0.001    0.000    0.367    0.007 readers.py:1732(read)
        1    0.002    0.002    0.309    0.309 modeling_utils.py:20(prep_wide_data_for_inference)
       50    0.001    0.000    0.301    0.006 c_parser_wrapper.py:222(read)
       50    0.282    0.006    0.285    0.006 {method 'read_low_memory' of 'pandas._libs.parsers.TextReader' objects}
    